<a href="https://colab.research.google.com/github/Dimildizio/DS_course/blob/main/Agents/Agentic_patterns/Pipeline_pattern.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prompt Chaining (Pipeline Pattern)

**Core AI pattern:**  transforms messy, unreliable one-shot prompts into robust pipelines.

Essential for building agents that can reason step by step, integrate tools, and manage context like humans do.

Building block for sophisticated workflows single prompts can't handle

## Theory

### What is it?
 Method of **breaking a big**, complex task **into smaller, sequential** prompts.

 Each step produces output that becomes the input for the next.

 Reduces errors, makes debugging easier, and allows models to handle tasks that would otherwise overwhelm them

### Why It Matters?

Using a single, **giant prompt for complex tasks often fails** because:

- Models lose track of instructions (instruction neglect).
- They drift from context (contextual drift).
- Errors in early steps propagate.
- Long prompts overload context windows.
- Cognitive load -> higher risk of hallucinations.

Prompt chaining solves this by **giving the model smaller, focused jobs.** Like dividing code into functions, each prompt has one role, one goal, and (ideally) a structured output (e.g., JSON).

### Key Components

- Sequential steps: each prompt does one task.
- Structured output: JSON ensures data passes cleanly between steps.
- External tools: chains can integrate APIs, databases, or calculators.

- Roles per step: e.g., "Summarizer," "Analyst", "Writer."

Frameworks: LangChain, LangGraph, Crew AI, Google ADK, Pydantic AI make chaining easier

### Practical Use Cases

- Information workflows: summarize -> extract entities -> query DB _. generate report.
- Complex queries: break into sub-questions, answer each, then synthesize.
- Data extraction: iterative invoice/receipt parsing with validation.
- Content generation: brainstorm → outline → draft → revise.
- Conversational agents: maintain dialogue state across multiple turns.
- Code generation: pseudocode → draft → static check → refine → document.
- Multimodal reasoning: combine text, images, and tables in steps

## TL;DR

**What**: Foundational pattern for Agentic AI systems. Single prompts overload LLMs -> unreliable outputs.

**Why**: Prompt chaining breaks tasks into clear, ordered steps → reliability + control.

**Rule of Thumb**: Use when tasks are too complex, require multiple stages, external tools, or multi-step reasoning.


### Key Takeaways

- Break problems into smaller steps (divide-and-conquer).
- Each step's output -> next step's input.
- Reliability UP, manageability UP.
- Use frameworksto formalize pipelines.



## Example

Let's consider malware analysis in our prompt example.

In [31]:
test_notes = """
2025-09-05 13:17 CET: SOC noted outbound traffic to 185.203.116.44 from srv-web-02.
Analyst claims: likely cobalt strike beacon; evidence: jittered 60s callbacks, /beacon path.
Artifacts: hash 9f2c...e12 (dll), domain acme-updates[.]com (first seen 2025-09-01).
Open questions: initial access vector unknown; user 'svc_iis' has unusual token privileges.
"""

### Set up

In [30]:
import os
import json
import time
import requests
from string import Template


from google.colab import userdata

In [47]:
OPENROUTER_API_KEY = userdata.get('openrouter')

BASE_URL = "https://openrouter.ai/api/v1/chat/completions"
HEADERS = {"Authorization": f"Bearer {OPENROUTER_API_KEY}"}
MODEL = "openrouter/sonoma-dusk-alpha"  # Because why not? We all know it's a cloaked xAI model, Elon


### Call and retry funcs

In [5]:
def call(model, messages, **kw):
    payload = {"model": model, "messages": messages, **kw}
    r = requests.post(BASE_URL, headers=HEADERS, json=payload, timeout=60)
    r.raise_for_status()
    return r.json()["choices"][0]["message"]["content"]

In [27]:
def safe_json(s, required_keys, max_retries=2):
    err = None
    for _ in range(max_retries+1):
        try:
            data=json.loads(s)
            if all(k in data for k in required_keys): return data
            err=f"Missing keys: {set(required_keys)-set(data)}"
        except Exception as e:
            err=str(e)
    raise ValueError(f"Invalid JSON after retries: {err}")


def call_json_with_retry(model, sys_prompt, user_prompt, required_keys, max_retries=2):
    msgs=[{"role":"system","content":sys_prompt},{"role":"user","content":user_prompt}]
    content = call(model, msgs)
    for i in range(max_retries+1):
        try:
            data = json.loads(content)
            if all(k in data for k in required_keys):
                return data
            missing = list(set(required_keys)-set(data))
            raise ValueError(f"Missing keys: {missing}")
        except Exception as e:
            if i == max_retries:
                raise
            # Nudge the model with the exact error
            msgs += [{"role":"assistant","content":content},
                     {"role":"user","content":f"You returned invalid JSON. Error: {e}. "
                                              f"Return ONLY valid JSON with keys {required_keys}."}]
            content = call(model, msgs)

### Single prompt result

In [32]:

SINGLE_SHOT_SYS = "You are an expert incident responder and technical writer."
SINGLE_SHOT_USER_T = Template("""You are given SOC notes:

$notes

Do ALL of the following in ONE response:
1) Summarize key findings in <=80 words.
2) Extract structured JSON with keys: "entities", "events" (each with "time","what"), "artifacts", "claims" ({"text":"","support":"weak|strong"}). Return ONLY VALID JSON (no prose) in a fenced ```json block.
3) Using those facts, write an executive brief with sections: TL;DR, Timeline, Indicators of Compromise, Analysis, Mitigations, Open Questions. <=350 words. Do not invent indicators not in the notes.
4) As a reviewer, output JSON: {"issues":[{"type":"unsupported|contradiction|format|missing","detail":""}], "overall_risk":"low|med|high"} pointing out problems in the draft.
5) Produce a FINAL revised brief that fixes ALL issues from step 4 (no new claims).
6) Draft an email to leadership (<=120 words) with next steps and owners.

Constraints:
- Perform steps strictly in order.
- Include BOTH JSON outputs AND all prose sections in one message.
- Use only information present in the notes.

Output format (follow exactly):
<<BEGIN>>
[Step 1 Summary]
[Step 2 JSON as ```json ... ```]
[Step 3 Executive Brief]
[Step 4 Review JSON]
[Step 5 Final Brief]
[Step 6 Email]
<<END>>
""")

In [33]:
def run_single_shot(model, notes):
    msgs = [
        {"role":"system","content":SINGLE_SHOT_SYS},
        {"role":"user","content":SINGLE_SHOT_USER_T.substitute(notes=notes)}
    ]
    return call(model, msgs)

In [34]:
single = run_single_shot(MODEL, test_notes)
print(single)

<<BEGIN>>
[Step 1 Summary]
SOC detected outbound traffic from srv-web-02 to 185.203.116.44 on 2025-09-05, suspected as Cobalt Strike beacon with 60s jittered callbacks and /beacon path. Artifacts include DLL hash 9f2c...e12 and domain acme-updates[.]com (first seen 2025-09-01). User 'svc_iis' shows unusual token privileges. Initial access vector unknown. (62 words)

[Step 2 JSON as ```json ... ```]
```json
{
  "entities": {
    "host": "srv-web-02",
    "user": "svc_iis",
    "ip": "185.203.116.44",
    "domain": "acme-updates[.]com"
  },
  "events": [
    {
      "time": "2025-09-05 13:17 CET",
      "what": "Outbound traffic from srv-web-02 to 185.203.116.44"
    },
    {
      "time": "2025-09-01",
      "what": "First seen domain acme-updates[.]com"
    }
  ],
  "artifacts": {
    "hash": "9f2c...e12 (dll)",
    "domain": "acme-updates[.]com"
  },
  "claims": [
    {
      "text": "likely cobalt strike beacon",
      "support": "strong"
    }
  ]
}
```

[Step 3 Executive Brief]

**

### Prompts

In [44]:
# prompt 1
EXTRACT_SYS = "You extract structured facts from noisy security notes."
EXTRACT_USER = """Notes:
{notes}
Return JSON: {{ "entities": [], "events": [{{"time":"", "what":""}}], "artifacts": [], "claims": [{{"text":"", "support":"weak|strong"}}] }}"""

# prompt 2
PLAN_SYS = "You are a planning assistant for a security brief."
PLAN_USER = """Given JSON facts:
{facts}
Plan: bullets for sections, what to verify, and missing info. Keep it short."""

# extra chains
# prompt 3
DRAFT_SYS = "You write concise executive security briefs for incident response leaders."
DRAFT_USER = """Using these facts and plan, draft the brief with sections:
TL;DR, Timeline, Indicators of Compromise, Analysis, Mitigations, Open Questions.
Facts JSON:
{facts}
Plan:
{plan}
Keep it <350 words, bullets where possible, no speculation."""

# prompt 4
CRITIC_SYS = "You are a strict incident-response reviewer. Be terse and specific."
CRITIC_USER = """Review the draft for: unsupported claims, contradictions with facts, formatting issues,
and missing critical mitigations. Return JSON:
{{"issues":[{{"type":"unsupported|contradiction|format|missing","detail":""}}], "overall_risk":"low|med|high"}}.
Draft:
{draft}
Facts:
{facts}"""


# prompt 5: final
FINALIZE_SYS = "You fix drafts precisely according to reviewer issues."
FINALIZE_USER = """Revise the draft to resolve ALL issues below. Do not add new claims.
Return only the final brief text.
Issues JSON:
{issues}
Draft:
{draft}"""

#### Funcs for chains

In [39]:
def run_extract(model, notes):
    m=[{"role":"system","content":EXTRACT_SYS},
       {"role":"user","content":EXTRACT_USER.format(notes=notes)}]
    return call(model, m)

def run_plan(model, facts_json):
    m=[{"role":"system","content":PLAN_SYS},
       {"role":"user","content":PLAN_USER.format(facts=facts_json)}]
    return call(model, m)

#### Extra funcs for extra chains

In [40]:
def run_draft(model, facts, plan):
    m=[{"role":"system","content":DRAFT_SYS},
       {"role":"user","content":DRAFT_USER.format(facts=json.dumps(facts), plan=plan)}]
    return call(model, m)

def run_critic(model, draft, facts):
    m=[{"role":"system","content":CRITIC_SYS},
       {"role":"user","content":CRITIC_USER.format(draft=draft, facts=json.dumps(facts))}]
    return call(model, m)

def run_finalize(model, draft, issues_json):
    m=[{"role":"system","content":FINALIZE_SYS},
       {"role":"user","content":FINALIZE_USER.format(issues=issues_json, draft=draft)}]
    return call(model, m)


### Try it!

Basic 2-step approach

In [51]:
# 1) Extract
extract_raw = run_extract(MODEL, test_notes)
print("EXTRACT (raw):\n", extract_raw)

facts = call_json_with_retry(MODEL, EXTRACT_SYS, EXTRACT_USER.format(notes=test_notes),
                             required_keys=["entities","events","artifacts","claims"])
print("\nEXTRACT (parsed):\n", json.dumps(facts, indent=2))

# 2) Plan
plan = run_plan(MODEL, json.dumps(facts))
print("\nPLAN:\n", plan)

EXTRACT (raw):
 {
  "entities": [
    {
      "type": "ip",
      "value": "185.203.116.44",
      "description": "Destination IP for outbound traffic"
    },
    {
      "type": "hostname",
      "value": "srv-web-02",
      "description": "Source server for outbound traffic"
    },
    {
      "type": "user",
      "value": "svc_iis",
      "description": "User account with unusual token privileges"
    },
    {
      "type": "domain",
      "value": "acme-updates.com",
      "description": "Suspicious domain first seen 2025-09-01"
    }
  ],
  "events": [
    {
      "time": "2025-09-05 13:17 CET",
      "what": "Outbound traffic detected from srv-web-02 to 185.203.116.44"
    }
  ],
  "artifacts": [
    {
      "type": "hash",
      "value": "9f2c...e12",
      "description": "Hash of suspicious DLL file"
    },
    {
      "type": "path",
      "value": "/beacon",
      "description": "Path associated with jittered callbacks"
    }
  ],
  "claims": [
    {
      "text": "Likely Co

### Try it!

Longer chain with draft, critic, and finalizer

In [28]:
draft = run_draft(MODEL, facts, plan)
print("\nDRAFT:\n", draft)

critic_raw = run_critic(MODEL, draft, facts)
print("\nCRITIC (raw):\n", critic_raw)

critic = safe_json(critic_raw, required_keys=["issues","overall_risk"])  # parse what we got

final_brief = run_finalize(MODEL, draft, json.dumps(critic))
print("\nFINAL:\n", final_brief)



DRAFT:
 ### TL;DR
- Suspicious outbound traffic from internal server srv-web-02 (user svc_iis) to IP 185.203.116.44 detected 2025-09-05, linked to domain acme-updates[.]com (first seen 2025-09-01).
- Strong evidence of Cobalt Strike beacon via jittered 60s callbacks to /beacon path; DLL hash 9f2c...e12 identified.
- Immediate verification and isolation recommended for srv-web-02.

### Timeline
- 2025-09-01: Domain acme-updates[.]com first observed.
- 2025-09-05 13:17 CET: SOC detects outbound traffic from srv-web-02 to 185.203.116.44.

### Indicators of Compromise
- IP: 185.203.116.44 (outbound destination).
- Hostname: srv-web-02 (source server).
- User: svc_iis (unusual token privileges).
- Domain: acme-updates[.]com (suspicious, tied to callbacks).
- Hash: 9f2c...e12 (DLL artifact).
- Path: /beacon (in jittered callbacks).

### Analysis
- Claim: Likely Cobalt Strike beacon (strong support from jittered 60s callbacks and /beacon path).
- Entity validation: Confirm IP reputation via 

### Evaluate final

In [29]:
score = 0
score += 1 if "TL;DR" in final_brief else 0
score += 1 if "Mitigations" in final_brief else 0
score += 1 if any(i["type"]=="unsupported" for i in critic["issues"]) or any(critic["issues"]) else 0
print("Heuristic score (0-3):", score)


Heuristic score (0-3): 3
